# Installs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install -q sentence-transformers segeval nltk corus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.2 MB/s eta 0:00:00


In [ ]:
!pip install -q pytorch-lightning pathlib2 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [ ]:
%load_ext autoreload
%autoreload 2

# Training

## Load git project

In [ ]:
import sys
nse_topseg_path = './NSE-TopicSegmentation'
!rm -rf {nse_topseg_path}
!git clone https://github.com/tony-pitchblack/NSE-TopicSegmentation.git
sys.path.append(nse_topseg_path)

Cloning into 'NSE-TopicSegmentation'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 368 (delta 78), reused 83 (delta 41), pack-reused 245
Receiving objects: 100% (368/368), 340.41 KiB | 403.00 KiB/s, done.
Resolving deltas: 100% (230/230), done.


## Load choi

In [ ]:
!rm -rf /content/text-segmentation
!git clone https://github.com/koomri/text-segmentation.git
!rm -rf ./data
!mkdir -p ./data/choi
!mv /content/text-segmentation/data/choi ./data/

Cloning into 'text-segmentation'...
remote: Enumerating objects: 1036, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1036 (delta 14), reused 9 (delta 9), pack-reused 1012
Receiving objects: 100% (1036/1036), 4.91 MiB | 7.09 MiB/s, done.
Resolving deltas: 100% (18/18), done.


## Train

In [ ]:
import os
def delete_experiment(exp_name):
    if os.path.exists(exp_name):
        ans = input(f"Experiment '{exp_name}' already exists. Delete [y/n]?")
        if ans.lower() == 'y':
            !rm -rf {exp_name}
    else:
        print(f"Experiment doesn't exist.")

In [ ]:
filepath = './NSE-TopicSegmentation/train_fit.py'
! python {filepath} --help

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-06-13 09:42:24.798689: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 09:42:24.798761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 09:42:24.950282: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 09:42:25.262913: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructio

In [ ]:
import shutil

filepath = './NSE-TopicSegmentation/train_fit.py'

! chmod 755 {filepath}
encoder_full_name = "cointegrated/rubert-tiny2"
dataset_name = 'lenta'
exp_name = 'lenta_test'
delete_experiment(exp_name)

! python {filepath} \
    --verbose \
    --dataset {dataset_name} --corus \
    -exp {exp_name} \
    --wandb --wandb_key aee284a72205e2d6787bd3ce266c5b9aefefa42c \
    --metric='F1' \
    -arc='BiLSTM' \
    --encoder={encoder_full_name} \
    --hidden_units=256 --num_layers=2 \
    --dropout_in 0.3 --dropout_out=0.3 \
    -lr=0.001 -bs=32 \
    --patience=5 \
    --threshold=0.5 \
    --max_epochs=1 \
    --save_embeddings \
    --max_docs_cnt=10 \
    -aux -ra
    # --max_docs_frac=0.5

Experiment doesn't exist.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-06-13 10:02:30.246934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 10:02:30.246999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 10:02:30.249344: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 10:02:30.261751: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized 

In [ ]:
SAVE_PATH = 'drive/MyDrive/practice/news-segmentation/'
subdirs = ['experiments', 'embeddings']
for subdir in subdirs:
    path = os.path.join(SAVE_PATH, subdir)
    os.makedirs(path, exist_ok=True)

In [ ]:
# !rm -rf { os.path.join(SAVE_PATH, 'experiments')}

In [ ]:
# !rm -rf {EXP_PATH}

In [ ]:
import shutil

EXP_PATH = os.path.join(SAVE_PATH, 'experiments', exp_name)
exp_src = exp_name
# shutil.copytree(src, EXP_PATH, dirs_exist_ok=True)
!cp {exp_src} {EXP_PATH}

EMB_PATH = os.path.join(SAVE_PATH, 'embeddings', exp_name)
src = os.path.join('embeddings', dataset_name)
shutil.copytree(src, EMB_PATH, dirs_exist_ok=True)

In [ ]:
from google.colab import runtime
runtime.unassign()

# Inference

## Common funcs

In [ ]:
def recover_segments(sentences, seg_bounds):
    segments = []
    last_bound = 0

    # append segments from 0 to n-1
    for sent_idx, (is_bound) in enumerate(seg_bounds):
        if is_bound:
            segment = sentences[last_bound:sent_idx+1]
            segments.append(segment)
            last_bound = sent_idx+1

    # append last segment
    segment = sentences[last_bound:]
    segments.append(segment)

    return segments

In [ ]:
from pprint import pprint

def print_segments(sections, headings=None, scores=None):
    print(f"SECTION COUNT: {len(sections)}")
    for i, section in enumerate(sections):
        print(f'\n-- SECTION {i+1} START --')
        if headings is not None:
            print(f'-- HEADING: {headings[i]}')
        pprint(section)
        print('-- SECTION END --')

## Infer on lenta

### Load dataloader

In [ ]:
from utils.load_datasets import load_dataset

max_docs_count = 10000
ds_corus = load_dataset('ria', corus=True, max_docs_cnt=max_docs_count, segments_per_doc=4)

Loading <utils.corus_dataset.CorusDataset object at 0x7b7151177f10> dataset...
Restricted to 10000 documents


In [ ]:
# TODO: remove sent_tags and iterate over ds_corus in SentenceDataset
sent_tags = []
for sent, tags, _ in ds_corus[0][1]:
    sent_tags.append([sent, tags])

In [ ]:
from models.EncoderDataset import SentenceDataset
ds_corus_train = ds_corus[0]
tag_to_ix = {0:0, 1:1, '<START>':2, '<STOP>':3}
sd_corus = SentenceDataset(sent_tags, tag_to_ix, encoder="cointegrated/rubert-tiny2")

Computing embeddings on cuda.


Computing embeddings: 100%|██████████| 2000/2000 [00:42<00:00, 46.57it/s]

Embeddings computation | total processing time: 0:00:43


In [ ]:
sd_corus[0]['embeddings'].shape

torch.Size([40, 312])

In [ ]:
from torch.utils.data import DataLoader
import torch

dl_corus = DataLoader(sd_corus, batch_size=8, collate_fn=sd_corus.collater)
batch = next(iter(dl_corus))

In [ ]:
batch.keys()

dict_keys(['id', 'src_tokens', 'src_lengths', 'tgt_tokens', 'src_sentences', 'src_segments'])

In [ ]:
batch['src_tokens'].shape

torch.Size([8, 101, 312])

### Load pretrained model

In [ ]:
from models.lightning_model import TextSegmenter
import os
import wandb

api = wandb.Api()
artifact = api.artifact('overfit1010/lenta_BiLSTM_F1/model-k4j7vuo7:v0', type='model')
art_dir = artifact.download()
ckpt_path = os.path.join(art_dir, 'model.ckpt')

text_seg_model = TextSegmenter.load_from_checkpoint(ckpt_path).to('cpu')
# key = aee284a72205e2d6787bd3ce266c5b9aefefa42c

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb:   1 of 1 files downloaded.  


In [ ]:
# from models.lightning_model import TextSegmenter
# import os

# SAVE_PATH = 'drive/MyDrive/practice/news-segmentation/'
# exp_name = 'lenta'
# ckpt_path = os.path.join(SAVE_PATH, 'experiments', exp_name, 'checkpoints', 'best_model.zip')

# text_seg_model = TextSegmenter.load_from_checkpoint(ckpt_path).to('cpu')

In [ ]:
scores, tags = text_seg_model.predict_step(batch, 0)

In [ ]:
text_seg_model.threshold = 0.5
text_seg_model.test_step(batch, 0)

### Eval

In [ ]:
from pytorch_lightning import Trainer

trainer = Trainer()
res = trainer.test(text_seg_model, dl_corus)
res

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          Pk_loss          │    0.07840225845575333    │
│          WD_loss          │    0.07840225845575333    │
│         test_loss         │            0.0            │
│         threshold         │            0.5            │
└───────────────────────────┴───────────────────────────┘

[{'Pk_loss': 0.07840225845575333,
  'WD_loss': 0.07840225845575333,
  'threshold': 0.5,
  'test_loss': 0.0}]

In [ ]:
from pytorch_lightning import Trainer

trainer = Trainer()
res = trainer.test(text_seg_model, dl_corus)
res

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          Pk_loss          │   0.045747946947813034    │
│          WD_loss          │    0.04803022742271423    │
│         test_loss         │    0.7580764293670654     │
│         threshold         │            0.5            │
└───────────────────────────┴───────────────────────────┘

[{'Pk_loss': 0.045747946947813034,
  'WD_loss': 0.04803022742271423,
  'threshold': 0.5,
  'test_loss': 0.7580764293670654}]

In [ ]:
batch.keys()

dict_keys(['id', 'src_tokens', 'src_lengths', 'tgt_tokens', 'src_sentences', 'src_segments'])

In [ ]:
doc_id = 3
pred_segments = recover_segments(batch['src_sentences'][doc_id], tags[doc_id])
tgt_segments = recover_segments(batch['src_sentences'][doc_id], batch['tgt_tokens'][doc_id])
len(pred_segments), len(tgt_segments)

(4, 10)

In [ ]:
scores[doc_id]

tensor([[1.0000e+00, 2.4145e-06],
        [1.0000e+00, 5.5856e-10],
        [1.0000e+00, 2.5986e-09],
        [1.0000e+00, 1.1311e-07],
        [1.0000e+00, 6.2319e-07],
        [1.0000e+00, 2.4155e-06],
        [9.9998e-01, 2.0885e-05],
        [9.9608e-01, 3.9197e-03],
        [1.0000e+00, 2.6255e-06],
        [9.9948e-01, 5.1526e-04],
        [1.0000e+00, 1.1822e-07],
        [1.0000e+00, 3.8485e-06],
        [1.0000e+00, 9.4278e-07],
        [1.0000e+00, 7.7326e-08],
        [9.9997e-01, 3.2898e-05],
        [1.0000e+00, 2.6482e-06],
        [1.0000e+00, 5.0131e-08],
        [1.0000e+00, 3.8660e-07],
        [1.0000e+00, 2.0825e-07],
        [5.0519e-03, 9.9495e-01],
        [1.0000e+00, 1.7973e-07],
        [1.0000e+00, 1.8157e-06],
        [1.0000e+00, 1.3111e-06],
        [1.0000e+00, 1.6350e-06],
        [1.0000e+00, 3.9074e-07],
        [1.0000e+00, 5.3169e-08],
        [1.0000e+00, 5.0207e-07],
        [1.0000e+00, 7.6814e-08],
        [1.0000e+00, 1.1057e-08],
        [1.174

In [ ]:
records = load_ria()

In [ ]:
print_segments(tgt_segments)

SECTION COUNT: 10

-- SECTION 1 START --
['российская теннисистка вера звонарева вышла в полуфинал турнира в чарльстоне '
 '(сша), призовой фонд которого составляет 700 тысяч долларов.',
 'в четвертьфинальном поединке звонарева, посеянная под 7-м номером, в двух '
 'сетах переиграла представительницу сша мелани удэн (13) - 7:5, 6:2.\n'
 'в полуфинале соперницей звонаревой станет первая ракетка соревнований '
 'датчанка каролин возняцки, победившая в пятницу другую россиянку - надежду '
 'петрову - 6:3 6:4. другую участницу финала определят саманта стосур '
 '(австралия, 4) и даниела хантукова (8).']
-- SECTION END --

-- SECTION 2 START --
['примите мои искренние соболезнования.',
 'ничто не может возместить ту утрату, которую вы понесли, но я уверен, что '
 'как сильный народ, вы найдете в себе силы и мужество не останавливаться и '
 'идти дальше.',
 'удачи.']
-- SECTION END --

-- SECTION 3 START --
['три гола шведского форварда хенрика зеттерберга принесли его "детройту" '
 'гостеву

In [ ]:
print_segments(pred_segments)

SECTION COUNT: 4

-- SECTION 1 START --
['российская теннисистка вера звонарева вышла в полуфинал турнира в чарльстоне '
 '(сша), призовой фонд которого составляет 700 тысяч долларов.',
 'в четвертьфинальном поединке звонарева, посеянная под 7-м номером, в двух '
 'сетах переиграла представительницу сша мелани удэн (13) - 7:5, 6:2.\n'
 'в полуфинале соперницей звонаревой станет первая ракетка соревнований '
 'датчанка каролин возняцки, победившая в пятницу другую россиянку - надежду '
 'петрову - 6:3 6:4. другую участницу финала определят саманта стосур '
 '(австралия, 4) и даниела хантукова (8).',
 'примите мои искренние соболезнования.',
 'ничто не может возместить ту утрату, которую вы понесли, но я уверен, что '
 'как сильный народ, вы найдете в себе силы и мужество не останавливаться и '
 'идти дальше.',
 'удачи.',
 'три гола шведского форварда хенрика зеттерберга принесли его "детройту" '
 'гостевую победу со счетом 7:4 над "финиксом" во втором матче первого раунда '
 'плей-офф в

## Infer on choi

In [ ]:
!rm -rf /content/text-segmentation
!git clone https://github.com/koomri/text-segmentation.git
!rm -rf ./data
!mkdir -p ./data/choi
!mv /content/text-segmentation/data/choi ./data/

Cloning into 'text-segmentation'...
remote: Enumerating objects: 1036, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1036 (delta 14), reused 9 (delta 9), pack-reused 1012
Receiving objects: 100% (1036/1036), 4.91 MiB | 11.93 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [ ]:
from utils.load_datasets import load_dataset

ds_choi = load_dataset('choi')

In [ ]:
a = ds_choi[0][0]
len(a)
len(a[0][0]) == len(a[0][1])

True

In [ ]:
from models.EncoderDataset import SentenceDataset

tag_to_ix = {0:0, 1:1, '<START>':2, '<STOP>':3}
sd = SentenceDataset(ds_choi[0][0], tag_to_ix, encoder="cointegrated/rubert-tiny2",
                     infer=False, CRF=True)

In [ ]:
from torch.utils.data import DataLoader

dl = DataLoader(sd, batch_size=2, collate_fn=sd.collater)
batch = next(iter(dl))

In [ ]:
batch.keys()

dict_keys(['id', 'src_tokens', 'src_lengths', 'tgt_tokens', 'src_sentences', 'src_segments'])

In [ ]:
from models.lightning_model import TextSegmenter

ckpt_path = '/content/lenta/checkpoints/checkpoint-epoch=00-valid_loss=0.00-threshold=0.00.ckpt'
text_seg_model = TextSegmenter.load_from_checkpoint(ckpt_path)

In [ ]:
%pdb off

Automatic pdb calling has been turned OFF


In [ ]:
batch["tgt_tokens"]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
%pdb on

Automatic pdb calling has been turned ON


In [ ]:
text_seg_model.test_step(batch, 0)

SegmentationMetricError: Reference and hypothesis segmentations differ in position length (191 is not 7).

> /usr/local/lib/python3.10/dist-packages/segeval/window/pk.py(33)__pk__()
     31     # Check for input errors
     32     if len(reference) != len(hypothesis):
---> 33         raise SegmentationMetricError(
     34             'Reference and hypothesis segmentations differ in position length ({0} is not {1}).'.format(len(reference), len(hypothesis)))
     35     # Compute window size to use if unspecified

ipdb> u
> /usr/local/lib/python3.10/dist-packages/segeval/util/__init__.py(81)__fnc_metric__()
     79             # Compare a single pair of segmentations
     80             del metric_kwargs['permuted']
---> 81             return fnc_metric(hypothesis, reference, **metric_kwargs)
     82     # Except if insufficient arguments supplied
     83     raise SegmentationMetricError('Incorrect arguments specified; expected 1 or 2, obtained {0} of value: {1}'.format(str(len(args)), str(args)))

ipdb> u
> /usr/local/lib/python3.10/dist-packages/segeval/window/pk.py(68)pk()
     64 
     